<img style="float: middle;" src="../images/logo.png">

# Resources

* The [Python 3 documentation]. For those unfamiliar with Python the [official tutorial] is recommended
* The Software Carpentry [novice Python lessons]
* [IPython's own notebook tutorial](http://nbviewer.jupyter.org/github/ipython/ipython/blob/3.x/examples/Notebook/Index.ipynb)
* [Markdown cheatsheet] (Markdown is the syntax you use to write formatted text into cells in a notebook.)

[Python 3 documentation]: https://docs.python.org/3/
[official tutorial]: https://docs.python.org/3/tutorial/index.html
[novice python lessons]: http://swcarpentry.github.io/python-novice-inflammation/
[Markdown cheatsheet]: https://github.com/adam-p/markdown-here/wiki/Markdown-Here-Cheatsheet

Often, there is a need to have a common analysis framework that allows the broad research community to use as a starting point for the analysis to establish baseline results that can be comparable across research groups, projects and technologies. Scanpy (Python) and Seurat (R) are such frameworks for single cell analysis. Importantly, it is necessary to understand the background theory and the low-level codes behind convenient wrapper functions.

For spatial transcriptomic data, there are many analysis types and more than 100s of software programs are available.

## 1. 10x Genomics Breast Cancer Samples

10x Genomics have made available samples from various tissue types, platforms and species. To get an idea of transcriptomic data and available analysis techniques, [we're going to use Visium datasets of human breast cancer samples](https://www.10xgenomics.com/datasets?query=&page=1&configure%5BhitsPerPage%5D=50&configure%5BmaxValuesPerFacet%5D=1000&refinementList%5Bplatform%5D%5B0%5D=Visium%20Spatial&refinementList%5BanatomicalEntities%5D%5B0%5D=breast).

For understanding about the Visium data, you could consult the lecture note. A three-minute introduction video can be found [here](https://www.youtube.com/watch?v=VwNk4d-0RJc). The diagram below explains the concept
![Visium Gene Expression Slide](https://ngisweden.scilifelab.se/wp-content/uploads/2021/02/Visium-gene-expression-slide-1024x390.png)

Visium data sets include:
* Images of the slides,
* Spatial location of spots, 55 𝜇m spot diameter and 100 𝜇m center to center, used to sample the transcriptome, and
* Gene expression counts, 1-10% of the total mRNA molecules present in a given spot, of the full transcriptome (~20,000 genes).

In [ ]:
import pathlib
import anndata
import matplotlib
import matplotlib.pyplot as matplotlib_pyplot
import numpy as numpy
import pandas as pandas
import scanpy as scanpy
import scipy as scipy
import sklearn.cluster as sklearn_cluster
import sklearn.decomposition as sklearn_decomposition
import sklearn.model_selection as sklearn_model_selection
import sklearn.preprocessing as sklearn_preprocessing
import tensorflow as tf
import tensorflow.keras as tf_keras
import tensorflow.keras.applications as tf_keras_applications
import tensorflow.keras.layers as tf_keras_layers
import tensorflow.keras.models as tf_keras_models
import umap
from requests.packages import target

In [ ]:
# Setup DPI and image output
dpi = 96
matplotlib.rcParams['figure.dpi'] = dpi
scanpy.settings.set_figure_params(dpi=dpi)

In [ ]:
# Formating for the notebook
# Suppress warnings for tidy representation of the notebook   
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
def print_msg(msg_lst: list, sep='\t', color='\033[91m'):
    msg = ""
    for i in msg_lst:
        msg += str(i) + sep

    print(color + "-" * 80 + bcolors.ENDC)
    print(color + msg.center(80, ' ') + bcolors.ENDC)
    print(color + '-' * 80 + bcolors.ENDC)

In [ ]:
# specify PATH to data
# Breast Cancer Sample from 10x Genomics
data_path = pathlib.Path("../data")
data_path.mkdir(parents=True, exist_ok=True)
results_path = pathlib.Path("../results")
results_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load Data
# The files are available at https://www.10xgenomics.com/datasets/human-breast-cancer-block-a-section-1-1-standard-1-1-0
# Specifically:
# * https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_spatial.tar.gz
# * https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_1/V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.tar.gz
library_id = 'V1_Breast_Cancer_Block_A_Section_1'
ann_data = scanpy.datasets.visium_sge(sample_id=library_id)

In [ ]:
raw = ann_data.copy()

### 1.1 Visualising Counts and Genes

<span style="color:blue">**Example problem**: Calculate the total counts (or reads) for each spot and store in '.obs['total_counts']. Next, calculate the number of genes detected (non-zero counts) and store in '.obs['n_genes_by_counts']. Then visualise the results, what do they mean?</span>

In [ ]:
ann_data.obs['total_counts'] = ann_data.X.sum(axis=1)
ann_data.obs['n_genes_by_counts'] = (ann_data.X > 0).sum(axis=1)

In [ ]:
# visualise metrics
scanpy.pl.spatial(ann_data, img_key="lowres", color=["total_counts", "n_genes_by_counts"])

Visualise H&E Image

A H&E ([Hematoxylin](https://en.wikipedia.org/wiki/Haematoxylin) and [Eosin](https://en.wikipedia.org/wiki/Eosin)) stain uses two dyes to stain a tissue section and provides important information about its structure. Hematoxylin stains cell nuclei blue-purple and eosin stains the cytoplasm and extracellular matrix in various shades of pink. Taken separately to the other data, it can be challenging to align it. [Fiducials](https://www.10xgenomics.com/support/software/space-ranger/latest/analysis/inputs/image-fiducial-alignment) are reference markers or points used for calibration, alignment, and orientation in imaging and spatial technologies. In spatial transcriptomics, they serve as fixed points of reference to align different types of data or images.

In [ ]:
scanpy.pl.spatial(ann_data, color=None, show=True)

### 2. Filtering and Normalisation

In a spatial dataset, each spot has its own spatial barcode, and genes are measured separately for each spot. Due to experimental limitations, biases can occur, where some spots receive more sequencing information than others, leading to variation in the total number of reads per spot. To address this technical variation, spot-based normalization is applied.

#### 2.1. Filtering

Filtering steps are commonly used in single-cell and spatial transcriptomics data preprocessing to help reduce these biases, technical noise reduction and to improved downstream processing results (normalisation, dimension reduction, differernt gene analysis, etc).

The following is relatively conservative and a better approach might be to observe the distribution of the data first and adjust values accordingly.

In [ ]:
ann_data.var_names_make_unique()
scanpy.pp.filter_genes(ann_data, min_cells=3)
scanpy.pp.filter_cells(ann_data, min_genes=10)

#### 2.2. Normalisation

Normalisation is an important step to address the challenges of varying RNA content and sequencing depth across different spots in a tissue section. Median normalisation adjusts gene expression values by scaling each spot's counts to a common median, effectively accounting for technical variations while preserving biological differences.

To achieve this, you need to calculate the total number of reads (or UMIs) for each spot and calculate the median across all spots in the dataset.

### 2.3. Visualising the Effects of Normalisation

We can observe the effect of the normalisation step. Due to spots having very similar values (near identical), the color scale bar is limited.

Here, we use the default normalization option in Scanpy, called "normalize by total." After normalization, each cell's total count is scaled to the median of the total counts across all cells before normalization. While this method is fast, it may be less accurate in certain biological scenarios.

In [ ]:
# Visualise before Normalisation
scanpy.pl.spatial(ann_data, img_key="lowres", color=["total_counts", "n_genes_by_counts"])

In [ ]:
# normalize and log transform data
scanpy.pp.normalize_total(ann_data, target_sum=1e4)
scanpy.pp.log1p(ann_data)

In [ ]:
# Update counts and number of genes
ann_data.obs['total_counts'] = ann_data.X.sum(axis=1)
ann_data.obs['n_genes_by_counts'] = (ann_data.X > 0).sum(axis=1)

In [ ]:
# Visualise After
scanpy.pl.spatial(ann_data, img_key="lowres", color=["total_counts", "n_genes_by_counts"])

# 3. Neural Networks

Neural networks are a type of machine learning model inspired by the structure and function of the human brain. They consist of interconnected nodes, or "neurons," organized in layers that process and transmit information.
Training data plays a crucial role in developing effective neural networks. It consists of labeled examples that the network uses to learn patterns and relationships, adjusting its internal parameters through a process called backpropagation to improve its performance on the given task.

To setup our neural network, first we're preparing the training and testing data sets: 70% of the data (x_train) is used for training, while 30% (x_test) is reserved for testing (evaluation). Later we will also assess KFold cross validation design to understand how random sampling can be used to train a neural network model (this part is optional).

An important task for a neural network is to learn features in the data. In the original data space, each gene represents a dimension. Through Neural Network (e.g., Autoencoder or Resnet50 as you will apply in this tutorial) we can find higher representation of the cell/spot in latent/reduced dimension space, where each dimension can be computed but is unknown in the context of which genes or which information they represent (e.g., a principal component in PCA or a vector of neurons in Autoencoder or Resnet50 outputs).  The latent dimensions (or hidden layer in the neural network architecture) is set to 128.

In [ ]:
# get an overview of the data
nn_raw_data = ann_data.to_df()
nn_raw_data.describe()

Prepare inputs for the neural network, splitting the data to training set (70%) and test set (30%)

In [ ]:
x_train, x_test = sklearn_model_selection.train_test_split(nn_raw_data, test_size=0.3, random_state=1)
feature_dim = len(nn_raw_data.columns)
latent_dim = 128

In [ ]:
print("Dimensions of x_train:", x_train.shape)
print("Dimensions of x_test:", x_test.shape)

# Aim and Approach

Aim: Integrating imaging features and sequencing features to latent space 

Approach: we will run three steps and explore two neural networks
1. Extract gene expression features using an autoencoder
2. Extract imaging features using CNN and transfer learning
3. Perform clustering and compare results from imaging alone, gene expression alone and the combination of both

# 3.1. Autoencoders
![Basic Diagram for an Autoencoder from Towards AI](../images/autoencoder.png)

From: ["Let’s Talk Auto-Encoders"](https://pub.towardsai.net/lets-talk-auto-encoders-d29e9a94e994)

An autoencoder is an unsupervised, neural network architecture. An encoder compresses the input data into a lower-dimensional representation called the "latent space" or "bottleneck". The decoder attempts to reconstruct the original input from this compressed representation. The bottleneck layer learns the most important features of the data, effectively creating a compact, informative representation. By comparing the reconstructed output to the original input, the autoencoder can be trained to minimise the reconstruction error, ensuring that the latent space captures the essential characteristics of the data while filtering out noise and less important details.

# 3.1.1  Model Architecture

In the code below, feature_dim represents the number of input features, determined by counting the columns in the original dataset (nn_raw_data) - in this case the number of genes. Latent_dim is the number of reduced dimensions the model will learn to represent the original data.

In [ ]:
class SpatialAutoencoder(tf_keras_models.Model):
    def __init__(self, input_dim, latent_dim):
        super(SpatialAutoencoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        self.encoder = tf_keras_models.Sequential([
            tf_keras_layers.InputLayer(input_shape=(input_dim,)),
            tf_keras_layers.Dense(128, activation='relu'),
            tf_keras_layers.Dense(64, activation='relu'),
            tf_keras_layers.Dense(latent_dim, activation='relu')
        ])

        self.decoder = tf_keras_models.Sequential([
            tf_keras_layers.InputLayer(input_shape=(latent_dim,)),
            tf_keras_layers.Dense(64, activation='relu'),
            tf_keras_layers.Dense(128, activation='relu'),
            tf_keras_layers.Dense(input_dim, activation='linear')
        ])

        # Create the full autoencoder model
        input_layer = tf_keras_layers.Input(shape=(input_dim,))
        encoded = self.encoder(input_layer)
        decoded = self.decoder(encoded)
        self.model = tf_keras_models.Model(inputs=input_layer, outputs=decoded)

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def get_latent_representation(self, x):
        return self.encoder(x)

In Neural Network, the loss function (also called objective function) is very important to optimise the model parameters in a way that it can perform the prediction most accurately by minimizing the loss between the expected values and the predicted values. Different loss functions have their own effects in how the model learns. For example, Mean Squared Error is suitable for a regression task while entropy loss is suitable for classification. Mean Squared Error is not as sensitive as Mean Absolute Error in detecting small difference. 

The mean squared error (MSE) is a common metric used to measure the average squared difference between the predicted values (y_hat) and the actual values (y) in a regression problem. It is calculated by taking the sum of the squared differences and dividing it by the number of samples (n):

MSE = (1/n) * sum((y_i - y_hat_i)^2)

The mean absolute error (MAE) is another common metric used to measure the average absolute difference between the predicted values (y_hat) and the actual values (y) in a regression problem. It is calculated by taking the sum of the absolute differences and dividing it by the number of samples (n):

MAE = (1/n) * sum(|y_i - y_hat_i|)

where:
- n is the number of samples
- y_i is the actual value of the i-th sample
- y_hat_i is the predicted value of the i-th sample


In [ ]:
loss_fn = tf_keras.losses.MeanSquaredError()

For the optimization of a neural network, gradient descend and back propagration are used (as explained in the lectorials). In the code, we are using the popular Adam optimizer. An introduction to Adam optimizer is here  more information, 

In [ ]:
autoencoder = SpatialAutoencoder(feature_dim, latent_dim)

In [ ]:
# Try different learning rates - you could consult the lecture note for the effect of learning rate.
# autoencoder.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=1e-3), loss=loss_fn)
autoencoder.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=1e-5), loss=loss_fn)

<span style="color:blue">**Example problem**: What is the total number of trainable parameters?</span>


In [ ]:
# Show a summary of the model
autoencoder.encoder.summary()  # total params


def get_trainable_params(model):
    total_params = 0
    for layer in model.layers:
        for weight in layer.trainable_weights:
            total_params += tf.size(weight).numpy()
    return total_params


trainable_params = get_trainable_params(autoencoder.encoder)
print(trainable_params)

## 3.2. Training the Autoencoder Model 

For the actual training of the model, we define hyper parameters such as the number of epochs (the number of iterations), the batch_size (the number of data points to be used in each parallel training. The size of the minibatch is often limited by the RAM of the GPU), and whether or not the entire training data is randomly shuffled before each epoch (to avoid reliance on the order of the data and so to improve generalization) 

In [ ]:
training_history = autoencoder.fit(x_train, x_train,
                                   epochs=15,
                                   batch_size=16,
                                   shuffle=True,
                                   validation_data=(x_test, x_test))

## 3.3. Examining Loss

### Plotting Total Loss

A good way to assess if the model is learning and if overfitting or underfitting occurs is through visualising the loss values across epoches and comparing the changes for training and testing dataset

In [ ]:
matplotlib_pyplot.plot(training_history.history["loss"], label="Training Loss")
matplotlib_pyplot.plot(training_history.history["val_loss"], label="Validation Loss")
matplotlib_pyplot.legend()

### Plotting Loss per Spot

Above, we plot the total loss across the whole dataset. Here, we can also visualise loss per spot, as each spot receives its own predicted value. We also compare MSE and MAE loss as explained above.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Get the reconstructed outputs
reconstructed = autoencoder.predict(x_test)

# Calculate reconstruction error for each sample
mse_per_sample = numpy.mean((x_test - reconstructed) ** 2, axis=1)
mae_per_sample = numpy.mean(numpy.abs(x_test - reconstructed), axis=1)

# Print overall reconstruction error
overall_mse = mean_squared_error(x_test.values.flatten(), reconstructed.flatten())
overall_mae = mean_absolute_error(x_test.values.flatten(), reconstructed.flatten())

print(f"Overall Mean Squared Error: {overall_mse}")
print(f"Overall Mean Absolute Error: {overall_mae}")

# Plot the reconstruction errors
matplotlib_pyplot.figure(figsize=(12, 6))

# Plot MSE
matplotlib_pyplot.subplot(1, 2, 1)
matplotlib_pyplot.plot(mse_per_sample, label='MSE per sample')
matplotlib_pyplot.xlabel('Sample Index')
matplotlib_pyplot.ylabel('MSE')
matplotlib_pyplot.title('Mean Squared Error per Sample')
matplotlib_pyplot.legend()

# Plot MAE
matplotlib_pyplot.subplot(1, 2, 2)
matplotlib_pyplot.plot(mae_per_sample, label='MAE per sample')
matplotlib_pyplot.xlabel('Sample Index')
matplotlib_pyplot.ylabel('MAE')
matplotlib_pyplot.title('Mean Absolute Error per Sample')
matplotlib_pyplot.legend()

matplotlib_pyplot.tight_layout()
matplotlib_pyplot.show()

## 3.4. Plotting Gene Expression Reconstruction from the Trained Autoencoder

After training the autoencoder, we have minimised the cost and are now able to reconstruct the expression pattern, where the reduced latent space should be able to be decoded to predict corresponding values. 

In [ ]:
# Assuming autoencoder, nn_raw_data, and ann_data are already defined
# Predict using Autoencoder
predicted_data = autoencoder.predict(ann_data.X)
print(nn_raw_data.shape)
print(predicted_data.shape)
print(ann_data.X.shape)

# Ensure the predicted data has the correct shape
if predicted_data.shape != ann_data.X.shape:
    raise ValueError(
        f"Shape mismatch: predicted_data has shape {predicted_data.shape}, but ann_data.X has shape {ann_data.X.shape}.")

# Write predicted_data to the X slot in ann_data
ann_data.X = predicted_data

# Ensure the predicted data has the correct shape before flattening
if predicted_data.shape[0] != ann_data.obs.shape[0]:
    raise ValueError(
        f"Length mismatch: predicted_data has length {predicted_data.shape[0]}, but ann_data.obs has length {ann_data.obs.shape[0]}.")

# Add nn_raw_data and predictions to ann_data.obs for visualization
ann_data.obs['predicted'] = predicted_data.mean(axis=1)
ann_data.obs['nn_raw'] = nn_raw_data.mean(axis=1)

In [ ]:
# Plot nn_raw_data and predictions on tissue
scanpy.pl.spatial(ann_data, img_key="lowres", color=["nn_raw", "predicted"], title="nn_raw_data on Tissue",
                  palette="Set2")

## 3.5. Testing on Unseen Datasets

The reconstruction in the session 1.4 above works well. However, one may question that the successful reconstruction is because the model was trained and used to predict the same dataset. This is a valid question, and so to address that we can try to use the trained autoencoder and test a different dataset. Here we will use a second breast cancer Visium dataset that is a replicate of the sample used in 1.4, but measured independently.

In [ ]:
# Load Data
# The files are available at https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_2/
# Specifically:
# * https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_spatial.tar.gz
# * https://cf.10xgenomics.com/samples/spatial-exp/1.1.0/V1_Breast_Cancer_Block_A_Section_2/V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.h5

ann_data_rep2 = scanpy.datasets.visium_sge(sample_id='V1_Breast_Cancer_Block_A_Section_2')

Reading in the data requires a h5 file and a spatial folder located in the same directory

In [ ]:
# Print the shape of the AnnData object to confirm successful loading
print(f"AnnData object created with shape: {ann_data_rep2.shape}")

# Visualize the spatial data
scanpy.pl.spatial(ann_data_rep2, color=None, show=True)

In [ ]:
ann_data_rep2.obs['total_counts'] = ann_data_rep2.X.sum(axis=1)
ann_data_rep2.obs['n_genes_by_counts'] = (ann_data_rep2.X > 0).sum(axis=1)

In [ ]:
# Check general information about the AnnData object
ann_data_rep2.obs

Preprocess this dataset in the same way

In [ ]:
raw2 = ann_data_rep2.copy()
ann_data_rep2.var_names_make_unique()
scanpy.pp.filter_genes(ann_data_rep2, min_cells=3)
scanpy.pp.filter_cells(ann_data_rep2, min_genes=10)
scanpy.pl.spatial(ann_data_rep2, img_key="lowres", color=["total_counts", "n_genes_by_counts"])

In [ ]:
# Print the shape of the input data
print(f"Shape of ann_data_rep2.X: {ann_data_rep2.X.shape}")

# Print the expected input shape of the autoencoder
expected_input_shape = autoencoder.model.input_shape[1:]  # Exclude the batch size
print(f"Expected input shape of the autoencoder: {expected_input_shape}")

# Ensure the data is of type float32
dense_data = ann_data_rep2.X.toarray() if not isinstance(ann_data_rep2.X, numpy.ndarray) else ann_data_rep2.X
if dense_data.dtype != numpy.float32:
    dense_data = dense_data.astype(numpy.float32)

# Adjust the input data shape if necessary
if dense_data.shape[1] != expected_input_shape[0]:
    print(f"Adjusting input data shape from {dense_data.shape[1]} to {expected_input_shape[0]}")
    if dense_data.shape[1] > expected_input_shape[0]:
        dense_data = dense_data[:, :expected_input_shape[0]]  # Truncate
    else:
        padding = expected_input_shape[0] - dense_data.shape[1]
        dense_data = numpy.pad(dense_data, ((0, 0), (0, padding)), 'constant')  # Pad with zeros

# Predict using the autoencoder
predicted_data = autoencoder.model.predict(dense_data)
print(f"Shape of predicted_data: {predicted_data.shape}")

# Add nn_raw_data and predictions to ann_data_rep2.obs for visualization
ann_data_rep2.obs['nn_raw'] = ann_data_rep2.X.mean(axis=1)
ann_data_rep2.obs['predicted'] = predicted_data.mean(axis=1)

In [ ]:
# Plot nn_raw and predicted on tissue
scanpy.pl.spatial(ann_data_rep2, img_key="lowres", color=["nn_raw", "predicted"], title="nn_raw_data on Tissue",
                  palette="Set2")

## 3.6. K-Fold Cross-Validation (Optional)

While training the autoencoder, we have assessed visually the loss in the train dataset and test dataset to see if the model is learning. Each time of random sampling would yield different results. We can assess the effect of this randomness using KFold cross validation. In many cases, after KFold runs, we can select the best performance model out of the K models to use.

In [ ]:
nn_raw_data_np = nn_raw_data.to_numpy()
kf = sklearn_model_selection.KFold(n_splits=3)
fold_no = 1
training_loss_per_fold = []
validation_loss_per_fold = []
autoencoder.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=1e-3), loss=loss_fn)
for train_index, test_index in kf.split(nn_raw_data_np):
    x_train_cv, x_test_cv = nn_raw_data_np[train_index], nn_raw_data_np[test_index]
    # Train the autoencoder
    training_history = autoencoder.fit(x_train_cv, x_train_cv,
                                       epochs=15,
                                       batch_size=16,
                                       shuffle=True,
                                       validation_data=(x_test_cv, x_test_cv))
    # Store training and validation loss
    training_loss_per_fold.append(training_history.history['loss'])
    validation_loss_per_fold.append(training_history.history['val_loss'])
    fold_no += 1
    autoencoder.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=1e-3), loss=loss_fn)
    # Plot the training and validation loss for each fold
matplotlib_pyplot.figure(figsize=(12, 8))
for i in range(len(training_loss_per_fold)):
    matplotlib_pyplot.plot(training_loss_per_fold[i], label=f'Training Loss Fold {i + 1}')
    matplotlib_pyplot.plot(validation_loss_per_fold[i], label=f'Validation Loss Fold {i + 1}', linestyle='--')
matplotlib_pyplot.title('Training and Validation Loss per Epoch for Each Fold')
matplotlib_pyplot.xlabel('Epoch')
matplotlib_pyplot.ylabel('Loss')
matplotlib_pyplot.legend()
matplotlib_pyplot.show()

# 4. Extracting Features from Imaging Data with Transfer Learning

Above, we use autoencoder to extract gene expression data. However, the imaging information is also important for cancer diagnosis. In fact, assessment of histological images by pathologists are used as gold standard diagnosis practice for many years (>100 years). Therefore, we now have the opportunity to incorporate the imaging information (what the eyes can see) with gene expression information (the eyes can not see) to make the best use of all information from a precious patient tissue sample. Below we will go through the steps to take.

### 4.1. Tiling Images

Images are often too large to conveniently process. A common approach in computer vision is to split (tile) the image and make use of the highly parallel computation to train the tiles. Notably, we also have corresponding locations of each spot that we measure the gene expression. To combine imaging information with the gene expression information for each spot, we will extract the tile image correspond to spot locations rather than using random tiling.

The following code require some understanding of the high resolution image vs low resolution image and scaling factors to convert the two resolutions and corresponding spot coordinates. This information is not essential so you could ignore the details in the code block below, just appreciate the concept that we extract the tiles based on their coordinate.

In [ ]:
# load high-resolution image 
highres_image = ann_data.uns['spatial'][library_id]['images']['hires']
# display the high-resolution image
matplotlib.pyplot.imshow(highres_image)
matplotlib.pyplot.axis('off')
matplotlib.pyplot.show()

In [ ]:
# Scale the spot coordinates to match the image dimensions
spot_coords = ann_data.obsm['spatial']
scalefactor = ann_data.uns['spatial'][library_id]['scalefactors']['tissue_hires_scalef']
spot_coords_scaled = spot_coords * scalefactor

# Function to extract tiles based on spot coordinates with boundary checks
def extract_tiles(image, coords, tile_size):
    tiles = []
    img_height, img_width, _ = image.shape
    half_tile = tile_size // 2
    for coord in coords:
        x, y = coord
        x_start = max(0, int(x - half_tile))
        y_start = max(0, int(y - half_tile))
        x_end = min(img_width, x_start + tile_size)
        y_end = min(img_height, y_start + tile_size)

        # Adjust start positions if the tile is smaller than the desired size
        if x_end - x_start < tile_size:
            x_start = x_end - tile_size
        if y_end - y_start < tile_size:
            y_start = y_end - tile_size

        tile = image[y_start:y_end, x_start:x_end]
        tiles.append(tile)
    return numpy.array(tiles)


# Define the tile size (adjust based on your data)
tile_size = 224  # Example tile size

# Extract tiles based on scaled spot coordinates
tiles = extract_tiles(highres_image, spot_coords_scaled, tile_size)

# Check the spot coordinates
print("Spot coordinates (first 10):", spot_coords[:10])
print("Scaled spot coordinates (first 10):", spot_coords_scaled[:10])

# Display some of the tiles
n = 3  # Number of tiles to display
matplotlib_pyplot.figure(figsize=(20, 4))
for i in range(n):
    ax = matplotlib_pyplot.subplot(1, n, i + 1)
    matplotlib_pyplot.imshow(tiles[i])
    matplotlib_pyplot.axis('off')
matplotlib_pyplot.show()

### 4.2. Transfer Learning

Transfer learning is a powerful tool in machine learning, where existing models have been trained on similar data or large, universal data can then be utilised in our new model. Here we use a model that has been trained on over one million images in the ImageNet dataset. The trained model is ResNet50. We can use this model to extract our image tiles. 

In [ ]:
# Preprocess the image tiles
def preprocess_image(image, target_size=(224, 224)):
    image = tf.image.resize(image, target_size)
    image = tf_keras_applications.resnet50.preprocess_input(image)
    return image


# Preprocess each tile
tiles_preprocessed = numpy.array([preprocess_image(tile) for tile in tiles])

# Add batch dimension
tiles_preprocessed = tf.convert_to_tensor(tiles_preprocessed, dtype=tf.float32)

# Load ResNet50 model without the top layer
base_model = tf_keras_applications.ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = tf_keras_layers.GlobalAveragePooling2D()(x)

# Add a fully connected layer with 128 neurons
x = tf_keras_layers.Dense(128, activation='relu')(x)

# Create the model
resnet_model = tf_keras_models.Model(inputs=base_model.input, outputs=x)

# Extract features from the tiles
resnet_features = resnet_model.predict(tiles_preprocessed)

print("Extracted ResNet50 features shape:", resnet_features.shape)
print("Extracted ResNet50 features:", resnet_features)

### 4.3. Clustering Based on Image Features


#### 4.3.1. Leiden Clustering


In [ ]:
# Create an AnnData object from the combined features
resnet_features_ann_data = anndata.AnnData(X=resnet_features)

# Perform compute nearest-neighbours index, leiden clustering, UMAP
scanpy.pp.neighbors(resnet_features_ann_data, n_neighbors=10, use_rep='X', random_state=42)
scanpy.tl.umap(resnet_features_ann_data)
scanpy.tl.leiden(resnet_features_ann_data, resolution=0.7, n_iterations=-1)

In [ ]:
# Create an AnnData object from the combined features
resnet_features_ann_data = anndata.AnnData(X=resnet_features)

# Perform compute nearest-neighbours index, leiden clustering, UMAP
scanpy.pp.neighbors(resnet_features_ann_data, n_neighbors=10, use_rep='X', random_state=42)
scanpy.tl.umap(resnet_features_ann_data)
scanpy.tl.leiden(resnet_features_ann_data, resolution=0.7, n_iterations=-1)

In [ ]:
# Display UMAP of features
scanpy.pl.umap(resnet_features_ann_data, color='leiden', cmap='Spectral')

In [ ]:
# Extract cluster labels
cluster_labels = resnet_features_ann_data.obs['leiden'].astype(int).to_numpy()
highres_image = ann_data.uns['spatial'][library_id]['images']['hires']
spot_coords = ann_data.obsm['spatial']

# Scale the spot coordinates to match the image dimensions
scalefactor = ann_data.uns['spatial'][library_id]['scalefactors']['tissue_hires_scalef']
spot_coords_scaled = spot_coords * scalefactor

# Plot clusters on tissue
matplotlib_pyplot.figure(figsize=(10, 10))
matplotlib_pyplot.imshow(highres_image, extent=[0, highres_image.shape[1], highres_image.shape[0], 0])
for i in numpy.unique(cluster_labels):
    cluster_spots = spot_coords_scaled[cluster_labels == i]
    matplotlib_pyplot.scatter(cluster_spots[:, 0], cluster_spots[:, 1], label=f'Cluster {i}', s=10)
matplotlib_pyplot.legend()
matplotlib_pyplot.title('Clusters on Tissue')
matplotlib_pyplot.axis('off')
matplotlib_pyplot.show()

In [ ]:
scanpy.pp.pca(ann_data, n_comps=50)
scanpy.pp.neighbors(ann_data, n_neighbors=20, random_state=42)
scanpy.tl.umap(ann_data)
scanpy.tl.leiden(ann_data, resolution=0.5, n_iterations=-1)
scanpy.pl.spatial(ann_data, img_key="lowres", color="leiden", title="Spatial distribution of Leiden clusters",
                  palette="Set2")

#### 4.3.2. K-means Clustering

In [ ]:
# Perform clustering
n_clusters = 6  # Adjust the number of clusters as needed
kmeans = sklearn_cluster.KMeans(n_clusters=n_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(resnet_features)

highres_image = ann_data.uns['spatial'][library_id]['images']['hires']
spot_coords = ann_data.obsm['spatial']

# Scale the spot coordinates to match the image dimensions
scalefactor = ann_data.uns['spatial'][library_id]['scalefactors']['tissue_hires_scalef']
spot_coords_scaled = spot_coords * scalefactor

# Plot clusters on tissue
matplotlib_pyplot.figure(figsize=(10, 10))
matplotlib_pyplot.imshow(highres_image, extent=[0, highres_image.shape[1], highres_image.shape[0], 0])
for i in range(n_clusters):
    cluster_spots = spot_coords_scaled[cluster_labels == i]
    matplotlib_pyplot.scatter(cluster_spots[:, 0], cluster_spots[:, 1], label=f'Cluster {i}', s=10)
matplotlib_pyplot.legend()
matplotlib_pyplot.title('Clusters on Tissue')
matplotlib_pyplot.axis('off')
matplotlib_pyplot.show()

## 5. Integrating Imaging and Sequencing Data

We can combine two key components of the data, namely tissue imaging and gene expression value. Each of our spot now has a latent representation from the autoencoder (for gene expression) and another representation from Resnet50 (for imaging data). We can then concatenate these two latent vectors (for each spot).

In [ ]:
# Extract latent space from the autoencoder
latent_space = autoencoder.encoder.predict(nn_raw_data)

print("Latent space shape:", latent_space.shape)
print("Latent space:", latent_space)

# Combine the features
combined_features = numpy.concatenate((resnet_features, latent_space), axis=1)

print("Combined features shape:", combined_features.shape)
print("Combined features:", combined_features)

The combined latent space now has information from both gene expression and imaging information. We can then perform clustering.

In [ ]:
# Assuming combined_features and ann_data are already defined
# Standardize the features
scaler = sklearn_preprocessing.StandardScaler()
combined_features_scaled = scaler.fit_transform(combined_features)

# Perform UMAP
umap_reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
umap_embedding = umap_reducer.fit_transform(combined_features_scaled)

# Convert UMAP embedding to AnnData object for Leiden clustering
combined_ann_data = scanpy.AnnData(combined_features_scaled)
combined_ann_data.obsm['X_umap'] = umap_embedding

# Compute the neighborhood graph
scanpy.pp.neighbors(combined_ann_data, n_neighbors=15, use_rep='X')

# Perform Leiden clustering
scanpy.tl.leiden(combined_ann_data, resolution=0.7, n_iterations=-1)

# Add Leiden clusters to ann_data
ann_data.obs['leiden_integrated'] = combined_ann_data.obs['leiden'].values

# Plot UMAP with Leiden clusters
matplotlib_pyplot.figure(figsize=(10, 8))
scanpy.pl.umap(combined_ann_data, color='leiden', show=False)
matplotlib_pyplot.title('UMAP projection with Leiden Clusters')
matplotlib_pyplot.show()

Examine clustering results on tissue. We can compare clustering of just using image as shown in 2.3.1 above vs combining image with the gene expression here and vs original.

In [ ]:
scanpy.pl.spatial(ann_data, img_key="lowres", color=["leiden", "leiden_integrated"], title=["Original Leiden Clusters", "Integrated Leiden Clusters"],
                  palette="Set2")